In [1]:
# imports
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import os

In [2]:
# define function to transform hourly to quarter hourly data
def hourly_to_quarter_hourly_df(df_h):
    # for each row, add three empty ones
    empty_rows = 4
    df_h.index = range(0, empty_rows*len(df_h), empty_rows)
    df_h = df_h.reindex(index=range(empty_rows*len(df_h)))

    # generate quarter_hourly interval series from df_h's start date on
    r = df_h.time[0].split(" ", 1)[0]
    day, month, year = int(r.split("-")[2]), int(r.split("-")[1]), int(r.split("-")[0])
    start_date = datetime(year,month,day)
    quarter_hourly = [start_date + timedelta(minutes=15*x) for x in range(0,len(df_h))]
    quarter_hourly_list = [x.strftime('%Y-%m-%d %H:%M') for x in quarter_hourly]
    # generate dataframe
    quarter_hourly_list = pd.DataFrame(quarter_hourly_list, columns=['time'])

    # insert quarter-hourly list into df_h -> df_qh
    df_h['time'] = quarter_hourly_list['time']
    df_qh = df_h[['time', 'electricity']]

    # interpolate hourly to quarter-hourly values: linear interpolation, then divide values by 4
    df_qh_output = df_qh.interpolate(method='linear')
    df_qh_output = df_qh_output.assign(electricity = lambda x: (x['electricity'] / 4))

    return df_qh_output

In [3]:
#
def generate_output_df(file_dir):
    # directory navigation
    os.chdir(file_dir)
    print("Current directory: {}".format(os.getcwd()))
    print("Generating df from {} files\n...".format(len(os.listdir())))

    # generate empty df
    file0 = os.listdir()[0]
    t0 = pd.read_csv(file0, sep=",", skiprows=3)
    t0 = t0[["time", "electricity"]]
    df = hourly_to_quarter_hourly_df(t0)[["time"]]

    # iterate over files and append data to df
    for file in os.listdir():
        # obtain string from file name
        s = file.split(".", 1)[0]
        # read in .csv
        t = pd.read_csv(file, sep=",", skiprows=3)
        t = t[["time", "electricity"]]
        # transform t from hourly to quarter_hourly and extract electricity data
        t_new = hourly_to_quarter_hourly_df(t)[["electricity"]]
        # append it to df
        df[s] = t_new

    # navigate back to parent dir
    os.chdir("..")
    print('Navigating back to parent folder:\n{}\n...'.format(os.getcwd()))

    return df

In [8]:
# write df to excel file
def print_df_to_excel(df, filename):
    print("Writing df to excel file {}.xlsx\n...".format(filename))
    df.to_excel("{}.xlsx".format(filename), index=False, header=True)
    print("Done.")
    return

In [5]:
# ----------------------------------------- ACTION SPACE -------------------------------------------------#
# specify path and navigate to files
root_dir = "C:\\Users\\joche\\ProgrammingProjects\\04 FIM-RC\\PV_wind power simulation"
# navigate to root dir
os.chdir(root_dir)
os.getcwd()

'C:\\Users\\joche\\ProgrammingProjects\\04 FIM-RC\\PV_wind power simulation'

In [10]:
# PV
df_pv = generate_output_df(file_dir="pv_data")
#df_pv
print_df_to_excel(df=df_pv, filename="pv_output")

Current directory: C:\Users\joche\ProgrammingProjects\04 FIM-RC\PV_wind power simulation\pv_data
Generating df from 16 files
...
Navigating back to parent folder:
C:\Users\joche\ProgrammingProjects\04 FIM-RC\PV_wind power simulation
...
Writing df to excel file pv_output.xlsx
...
Done.


In [9]:
# Wind
df_wind = generate_output_df(file_dir="wind_data")
#df_wind
print_df_to_excel(df=df_wind, filename="wind_output")

Current directory: C:\Users\joche\ProgrammingProjects\04 FIM-RC\PV_wind power simulation\wind_data
Generating df from 16 files
...
Navigating back to parent folder:
C:\Users\joche\ProgrammingProjects\04 FIM-RC\PV_wind power simulation
...
Writing df to excel file wind_output.xlsx
...
Done.
